In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [2]:
from jubilee_controller import JubileeMotionController
import matplotlib.pyplot as plt
import time
import numpy as np 
from pynput import keyboard
import cv2
from datetime import datetime
from camera_controller import camera_tool
from email_sender import send_email
from image_processing import detect_circle
from gripper_controller import Gripper
from micropipette_controller import Micropipette

In [4]:
jubilee = JubileeMotionController()

In [35]:
jubilee.home_all(mesh_mode_z=False)
jubilee.protect_tools(on=True)
jubilee.move_xyz_absolute(z=50)

In [13]:
pipeta = Micropipette(jubilee,parking_position_xy=(138,16),move_velocity=3000)

In [ ]:
pipeta.uninstall()

<p align ='justify'>
Para preparar a solução dos núcleos de ouro, 
300 μL de <em>HAuCl<sub>4</sub></em> (Sigma-Aldrich, ≥ 99.9%) 
0,01 M foram adicionados a 7,5 mL de 
<em>CTAB</em> (Sigma-Aldrich, ≥ 99.9%) 0,1 M sob agitação. 
Em seguida, foram adicionados 500 μL de 
<em>NaBH<sub>4</sub></em> (Sigma-Aldrich, ≥ 99.9%) 0,01 M resfriado 
e o sistema foi mantido sob agitação vigorosa por 10 minutos.
Após esse período, o sistema foi mantido em repouso à temperatura ambiente 
(23&nbsp;°C) e protegido da luz por 2 horas antes do uso.
</p>


In [21]:
posicao_erlenmeyer = (243,161,300)
posicao_descarte = (105,240,270)
posicao_caixa_ponteiras = [144.5,315.5,160]

acido_cloroaurico = (36,183,156)
ctab = (133,133,160)
borohidreto = (36,130,156)



In [22]:
altura_segura_1 = 280
altura_segura_2 = 320

#Acoplar Primeira Ponteira

jubilee.move_xyz_absolute(z=altura_segura_1)
jubilee.move_xyz_absolute(posicao_caixa_ponteiras[0],posicao_caixa_ponteiras[1])
jubilee.move_xyz_absolute(z=posicao_caixa_ponteiras[2],velocity=500)
jubilee.move_xyz_absolute(z=altura_segura_1)


#Pipeta 300ul de ácido cloroáurico

jubilee.move_xyz_absolute(acido_cloroaurico[0],acido_cloroaurico[1])
pipeta.press(300)
jubilee.move_xyz_absolute(z=acido_cloroaurico[2],velocity=500)
pipeta.aspirate()
jubilee.move_xyz_absolute(z=altura_segura_2)
jubilee.move_xyz_absolute(posicao_erlenmeyer[0],posicao_erlenmeyer[1])
jubilee.move_xyz_absolute(z=posicao_erlenmeyer[2],velocity=500)
pipeta.dispense(velocity=300)
jubilee.move_xyz_absolute(z=altura_segura_2)


#Remover ponteira

jubilee.move_xyz_absolute(posicao_descarte[0],posicao_descarte[1])
jubilee.move_xyz_absolute(z=posicao_descarte[2],velocity=1000)
pipeta.eject_tip()


#Acoplar Segunda Ponteira


posicao_caixa_ponteiras[0]-=10
jubilee.move_xyz_absolute(posicao_caixa_ponteiras[0],posicao_caixa_ponteiras[1])
jubilee.move_xyz_absolute(z=posicao_caixa_ponteiras[2],velocity=500)
jubilee.move_xyz_absolute(z=altura_segura_1)


#Pipeta 7500ul de CTAB


for i in range(7):

    jubilee.move_xyz_absolute(ctab[0],ctab[1])
    pipeta.press(1000)
    jubilee.move_xyz_absolute(z=ctab[2],velocity=500)
    pipeta.aspirate()
    jubilee.move_xyz_absolute(z=altura_segura_2)
    jubilee.move_xyz_absolute(posicao_erlenmeyer[0],posicao_erlenmeyer[1])
    jubilee.move_xyz_absolute(z=posicao_erlenmeyer[2],velocity=500)
    pipeta.dispense(velocity=50)
    jubilee.move_xyz_absolute(z=altura_segura_2)

jubilee.move_xyz_absolute(ctab[0],ctab[1])
pipeta.press(500)
jubilee.move_xyz_absolute(z=ctab[2],velocity=500)
pipeta.aspirate()
jubilee.move_xyz_absolute(z=altura_segura_2)
jubilee.move_xyz_absolute(posicao_erlenmeyer[0],posicao_erlenmeyer[1])
jubilee.move_xyz_absolute(z=posicao_erlenmeyer[2],velocity=500)
pipeta.dispense(velocity=50)
jubilee.move_xyz_absolute(z=altura_segura_2)


#Ejetar segunda ponteira

jubilee.move_xyz_absolute(posicao_descarte[0],posicao_descarte[1])
jubilee.move_xyz_absolute(z=posicao_descarte[2],velocity=1000)
pipeta.eject_tip()


#Acoplar Terceira Ponteira

posicao_caixa_ponteiras[0]-=10
jubilee.move_xyz_absolute(posicao_caixa_ponteiras[0],posicao_caixa_ponteiras[1])
jubilee.move_xyz_absolute(z=posicao_caixa_ponteiras[2],velocity=500)
jubilee.move_xyz_absolute(z=altura_segura_1)


#Pipeta 500ul de borohidreto de sódio

jubilee.move_xyz_absolute(borohidreto[0],borohidreto[1])
pipeta.press(500)
jubilee.move_xyz_absolute(z=borohidreto[2],velocity=500)
pipeta.aspirate()
jubilee.move_xyz_absolute(z=altura_segura_2)
jubilee.move_xyz_absolute(posicao_erlenmeyer[0],posicao_erlenmeyer[1])
jubilee.move_xyz_absolute(z=posicao_erlenmeyer[2],velocity=500)
pipeta.dispense(velocity=300)
jubilee.move_xyz_absolute(z=altura_segura_2)

#Ejetar terceira ponteira

jubilee.move_xyz_absolute(posicao_descarte[0],posicao_descarte[1])
jubilee.move_xyz_absolute(z=posicao_descarte[2],velocity=1000)
pipeta.eject_tip()


KeyboardInterrupt: 

In [23]:
pipeta.uninstall()